In [1]:
# Import the GLI library
import sys
import os

# Add the parent directory to the path to import gli
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'python'))

import gli
from gli import Graph, get_available_backends, set_backend, get_current_backend, create_random_graph
import time
import random

print("GLI Tutorial - Graph Operations Demo")
print(f"Available backends: {get_available_backends()}")
print(f"Current backend: {get_current_backend()}")

GLI Tutorial - Graph Operations Demo
Available backends: ['python', 'rust']
Current backend: rust


In [2]:
g = Graph()
    
# Create employees with predictable attributes for clear filtering
departments = ['Engineering', 'Sales', 'Marketing']
roles = ['Junior', 'Senior', 'Manager']

employee_ids = []

# Create 30000 employees with controlled attributes
for i in range(30000):
    # Create predictable patterns for clear filtering
    dept = departments[i % 3]
    
    # Make some employees clearly senior
    if i < 50:
        role = 'Manager'
        salary = random.randint(100000, 150000)
        performance = random.uniform(4.0, 5.0)
    elif i < 150:
        role = 'Senior'
        salary = random.randint(80000, 120000)
        performance = random.uniform(3.5, 4.5)
    else:
        role = 'Junior'
        salary = random.randint(50000, 80000)
        performance = random.uniform(3.0, 4.0)
    
    employee_id = g.add_node(
        name=f"Employee_{i:03d}",
        department=dept,
        role=role,
        salary=salary,
        performance_score=round(performance, 1),
        employee_id=i,
        is_remote=i % 4 == 0  # Every 4th person is remote
    )
    employee_ids.append(employee_id)

print(f"✅ Created graph: {g.node_count()} employees")

# Add some management relationships
for i in range(0, 500):  # First 500 are managers
    manager = employee_ids[i]
    # Each manager oversees 3-5 people
    for j in range(3):
        if i * 3 + j + 500 < len(employee_ids):
            report = employee_ids[i * 3 + j + 500]
            g.add_edge(manager, report, relationship='manages')

print(f"✅ Added management relationships: {g.edge_count()} edges")

# Commit initial state
print("\n💾 Committing initial state...")
initial_hash = g.commit("Initial company structure")
print(f"✅ Initial state: {initial_hash}")

# Initial filtering
print("\n🔍 Initial State Filtering:")

if g.use_rust:
    # Filter managers
    managers = g.filter_nodes_by_attributes({'role': 'Manager'})
    print(f"📊 Managers: {len(managers)}")
    
    # Filter engineering department
    engineers = g.filter_nodes_by_attributes({'department': 'Engineering'})
    print(f"📊 Engineering employees: {len(engineers)}")
    
    # Filter remote workers
    remote_workers = g.filter_nodes_by_attributes({'is_remote': True})
    print(f"📊 Remote workers: {len(remote_workers)}")
    
    # Filter high earners (>= 90k)
    high_earners = []
    for node_id in employee_ids:
        node = g.get_node(node_id)
        if node and node.attributes.get('salary', 0) >= 90000:
            high_earners.append(node_id)
    print(f"📊 High earners (≥$90k): {len(high_earners)}")

# Make significant changes
print("\n🔄 Making significant changes...")

changes_made = {
    'promotions': 0,
    'salary_bumps': 0,
    'new_remote': 0
}

# Promote some juniors to seniors
juniors = g.filter_nodes_by_attributes({'role': 'Junior'})
for i, node_id in enumerate(juniors[:30]):  # Promote first 30 juniors
    g.set_node_attribute(node_id, 'role', 'Senior')
    # Give them a salary bump
    current_node = g.get_node(node_id)
    if current_node:
        new_salary = int(current_node.attributes.get('salary', 50000) * 1.2)
        g.set_node_attribute(node_id, 'salary', new_salary)
    changes_made['promotions'] += 1

# Give raises to top performers
for node_id in employee_ids:
    node = g.get_node(node_id)
    if node and node.attributes.get('performance_score', 0) >= 4.0:
        current_salary = node.attributes.get('salary', 50000)
        new_salary = int(current_salary * 1.15)
        g.set_node_attribute(node_id, 'salary', new_salary)
        changes_made['salary_bumps'] += 1

# Make some employees remote
for i in range(0, 300, 7):  # Every 7th employee becomes remote
    if i < len(employee_ids):
        g.set_node_attribute(employee_ids[i], 'is_remote', True)
        changes_made['new_remote'] += 1

print(f"✅ Changes applied:")
print(f"   📈 Promotions: {changes_made['promotions']}")
print(f"   💰 Salary increases: {changes_made['salary_bumps']}")
print(f"   🏠 New remote workers: {changes_made['new_remote']}")

# Commit modified state
print("\n💾 Committing modified state...")
modified_hash = g.commit("Annual review - promotions and raises")
print(f"✅ Modified state: {modified_hash}")

# Modified state filtering
print("\n🔍 Modified State Filtering:")

if g.use_rust:
    # Same filters on modified state
    managers_mod = g.filter_nodes_by_attributes({'role': 'Manager'})
    print(f"📊 Managers: {len(managers_mod)}")
    
    seniors_mod = g.filter_nodes_by_attributes({'role': 'Senior'})
    print(f"📊 Seniors: {len(seniors_mod)}")
    
    engineers_mod = g.filter_nodes_by_attributes({'department': 'Engineering'})
    print(f"📊 Engineering employees: {len(engineers_mod)}")
    
    remote_workers_mod = g.filter_nodes_by_attributes({'is_remote': True})
    print(f"📊 Remote workers: {len(remote_workers_mod)}")
    
    # Count high earners again
    high_earners_mod = []
    for node_id in employee_ids:
        node = g.get_node(node_id)
        if node and node.attributes.get('salary', 0) >= 90000:
            high_earners_mod.append(node_id)
    print(f"📊 High earners (≥$90k): {len(high_earners_mod)}")
    
    # Show changes
    print(f"\n📈 Comparison:")
    print(f"   👔 Managers: {len(managers)} → {len(managers_mod)} ({len(managers_mod) - len(managers):+d})")
    print(f"   🎓 Seniors: ? → {len(seniors_mod)} (after promotions)")
    print(f"   🔧 Engineers: {len(engineers)} → {len(engineers_mod)} ({len(engineers_mod) - len(engineers):+d})")
    print(f"   🏠 Remote workers: {len(remote_workers)} → {len(remote_workers_mod)} ({len(remote_workers_mod) - len(remote_workers):+d})")
    print(f"   💰 High earners: {len(high_earners)} → {len(high_earners_mod)} ({len(high_earners_mod) - len(high_earners):+d})")
    
    # Show some specific examples
    print(f"\n👑 Sample promoted employees:")
    promoted_seniors = g.filter_nodes({'role': 'Senior'})
    for i, node_id in enumerate(promoted_seniors[:5]):
        node = g.get_node(node_id)
        if node:
            print(f"   • {node.attributes.get('name')}: {node.attributes.get('role')} in {node.attributes.get('department')} (${node.attributes.get('salary', 0):,})")
    
    # Storage stats
    stats = g.get_storage_stats()
    print(f"\n📊 Final storage stats: {stats}")



✅ Created graph: 30000 employees
✅ Added management relationships: 1500 edges

💾 Committing initial state...
✅ Initial state: 683eaad4aa340052

🔍 Initial State Filtering:
📊 Managers: 50
📊 Engineering employees: 10000
📊 Remote workers: 7500
📊 High earners (≥$90k): 120

🔄 Making significant changes...
✅ Changes applied:
   📈 Promotions: 30
   💰 Salary increases: 1575
   🏠 New remote workers: 43

💾 Committing modified state...
✅ Modified state: ca554ab1d40bc329

🔍 Modified State Filtering:
📊 Managers: 50
📊 Seniors: 130
📊 Engineering employees: 10000
📊 Remote workers: 7532
📊 High earners (≥$90k): 229

📈 Comparison:
   👔 Managers: 50 → 50 (+0)
   🎓 Seniors: ? → 130 (after promotions)
   🔧 Engineers: 10000 → 10000 (+0)
   🏠 Remote workers: 7500 → 7532 (+32)
   💰 High earners: 120 → 229 (+109)

👑 Sample promoted employees:
   • Employee_8841: Senior in Engineering ($67,911)
   • Employee_2024: Senior in Marketing ($84,471)
   • Employee_14648: Senior in Marketing ($99,905)
   • Employee_25575

In [3]:
g.nodes.items()[0:5]

[('node_8cdf6625',
  Node(id='node_8cdf6625', attributes={'role': 'Senior', 'name': 'Employee_8841', 'employee_id': 8841, 'is_remote': False, 'department': 'Engineering', 'performance_score': 3.8, 'salary': 67911})),
 ('node_09783a10',
  Node(id='node_09783a10', attributes={'performance_score': 3.5, 'employee_id': 2024, 'name': 'Employee_2024', 'salary': 84471, 'is_remote': True, 'department': 'Marketing', 'role': 'Senior'})),
 ('node_2cddfc2b',
  Node(id='node_2cddfc2b', attributes={'department': 'Marketing', 'salary': 99905, 'name': 'Employee_14648', 'is_remote': True, 'performance_score': 4.0, 'role': 'Senior', 'employee_id': 14648})),
 ('node_96010caa',
  Node(id='node_96010caa', attributes={'department': 'Engineering', 'salary': 60244, 'is_remote': False, 'performance_score': 3.1, 'role': 'Senior', 'name': 'Employee_25575', 'employee_id': 25575})),
 ('node_3d49635f',
  Node(id='node_3d49635f', attributes={'is_remote': True, 'salary': 70409, 'performance_score': 4.0, 'role': 'Senio

In [4]:
# Test lambda filtering - correct signature is (node_id, attributes)
marketing_employees = g.filter_nodes(lambda node_id, attrs: attrs.get('department') == 'Marketing')
print(f"Marketing employees: {len(marketing_employees)}")

# Test with multiple conditions
senior_engineers = g.filter_nodes(lambda node_id, attrs: 
                                  attrs.get('role') == 'Senior' and attrs.get('department') == 'Engineering')
print(f"Senior Engineers: {len(senior_engineers)}")

# Show some examples
print(f"\n🏢 Sample Marketing employees:")
for i, emp_id in enumerate(marketing_employees[:3]):
    node = g.get_node(emp_id)
    if node:
        print(f"   • {node.attributes.get('name')}: {node.attributes.get('role')} (${node.attributes.get('salary', 0):,})")

Marketing employees: 10000
Senior Engineers: 42

🏢 Sample Marketing employees:
   • Employee_2024: Senior ($84,471)
   • Employee_14648: Senior ($99,905)
   • Employee_17420: Senior ($70,409)


In [5]:
# Test both attribute-based and lambda-based filtering

print("🔍 Attribute-based filtering:")
# Using dictionary filters (old style still works)
engineering_dept = g.filter_nodes({'department': 'Engineering'})
remote_workers = g.filter_nodes({'is_remote': True})
print(f"   📊 Engineering dept: {len(engineering_dept)}")
print(f"   🏠 Remote workers: {len(remote_workers)}")

print("\n🔍 Lambda-based filtering:")
# Using lambda functions for more complex conditions
high_earners = g.filter_nodes(lambda node_id, attrs: attrs.get('salary', 0) >= 90000)
senior_remote = g.filter_nodes(lambda node_id, attrs: 
                              attrs.get('role') == 'Senior' and attrs.get('is_remote') == True)
print(f"   💰 High earners (≥$90k): {len(high_earners)}")
print(f"   👔 Senior remote workers: {len(senior_remote)}")

# Complex condition combining multiple attributes
high_performing_juniors = g.filter_nodes(lambda node_id, attrs: 
                                        attrs.get('role') == 'Junior' and 
                                        attrs.get('performance_score', 0) >= 3.5)
print(f"   ⭐ High-performing juniors: {len(high_performing_juniors)}")

print("\n📈 Performance comparison:")
print(f"   Traditional dict filter vs lambda: {len(engineering_dept)} == {len(g.filter_nodes(lambda nid, a: a.get('department') == 'Engineering'))}")
print("   ✅ Both methods work consistently!")

🔍 Attribute-based filtering:
   📊 Engineering dept: 10000
   🏠 Remote workers: 7532

🔍 Lambda-based filtering:
   💰 High earners (≥$90k): 229
   👔 Senior remote workers: 39
   ⭐ High-performing juniors: 16416

📈 Performance comparison:
   Traditional dict filter vs lambda: 10000 == 10000
   ✅ Both methods work consistently!
   ⭐ High-performing juniors: 16416

📈 Performance comparison:
   Traditional dict filter vs lambda: 10000 == 10000
   ✅ Both methods work consistently!


In [6]:
# 🕰️ Working with Previous States of the Graph

print("📚 Basic state operations:")
print(f"   Current hash: {getattr(g, 'current_hash', 'None')}")
print(f"   Available saved states:")
print(f"      • initial_hash = {initial_hash}")
print(f"      • modified_hash = {modified_hash}")

print(f"\n🔍 Current state stats:")
current_seniors = g.filter_nodes({'role': 'Senior'})
current_remote = g.filter_nodes({'is_remote': True})
print(f"   👔 Senior employees: {len(current_seniors)}")
print(f"   🏠 Remote workers: {len(current_remote)}")

# Let's check what state management methods are available
print(f"\n🔧 Available state methods:")
state_methods = [method for method in dir(g) if 'state' in method.lower() or 'branch' in method.lower()]
for method in state_methods:
    if not method.startswith('_'):
        print(f"   • {method}")

print(f"\n💾 Storage stats:")
stats = g.get_storage_stats()
for key, value in stats.items():
    print(f"   {key}: {value}")

📚 Basic state operations:
   Current hash: ca554ab1d40bc329
   Available saved states:
      • initial_hash = 683eaad4aa340052
      • modified_hash = ca554ab1d40bc329

🔍 Current state stats:
   👔 Senior employees: 130
   🏠 Remote workers: 7532

🔧 Available state methods:
   • auto_states
   • branch_heads
   • branches
   • create_branch
   • current_branch
   • get_state_info
   • load_state
   • max_auto_states
   • save_state
   • states

💾 Storage stats:
   branches: 1
   edge_refs_tracked: 1500
   total_states: 3
   node_refs_tracked: 31628
   pooled_nodes: 31628
   pooled_edges: 1500


In [7]:
# 🔄 Loading Previous States

print("🕰️ Time travel: Loading the initial state...")

# Method 1: Using load_state() if available
try:
    # Save current state hash for later
    current_state = modified_hash
    
    # Load the initial state
    if hasattr(g, 'load_state'):
        g.load_state(initial_hash)
        print(f"✅ Loaded initial state: {initial_hash}")
    else:
        print("⚠️ load_state method not available")
except Exception as e:
    print(f"❌ Error loading state: {e}")

# Check the data after loading initial state
print(f"\n🔍 After loading initial state:")
initial_seniors = g.filter_nodes({'role': 'Senior'})
initial_remote = g.filter_nodes({'is_remote': True}) 
initial_high_earners = g.filter_nodes(lambda nid, attrs: attrs.get('salary', 0) >= 90000)

print(f"   👔 Senior employees: {len(initial_seniors)}")
print(f"   🏠 Remote workers: {len(initial_remote)}")
print(f"   💰 High earners (≥$90k): {len(initial_high_earners)}")

print(f"\n📊 Comparison with modified state:")
print(f"   👔 Seniors: {len(initial_seniors)} (initial) vs {len(current_seniors)} (modified)")
print(f"   🏠 Remote: {len(initial_remote)} (initial) vs {len(current_remote)} (modified)")

🕰️ Time travel: Loading the initial state...
✅ Loaded initial state: 683eaad4aa340052

🔍 After loading initial state:
   👔 Senior employees: 100
   🏠 Remote workers: 7500
   💰 High earners (≥$90k): 120

📊 Comparison with modified state:
   👔 Seniors: 100 (initial) vs 130 (modified)
   🏠 Remote: 7500 (initial) vs 7532 (modified)
   👔 Senior employees: 100
   🏠 Remote workers: 7500
   💰 High earners (≥$90k): 120

📊 Comparison with modified state:
   👔 Seniors: 100 (initial) vs 130 (modified)
   🏠 Remote: 7500 (initial) vs 7532 (modified)


In [8]:
# 🎯 Complete State Management Workflow

print("🎯 Demonstrating complete state loading workflow:")

# Store current state info
current_seniors = len(g.filter_nodes({'role': 'Senior'}))
current_remote = len(g.filter_nodes({'is_remote': True}))
current_high_earners = len(g.filter_nodes(lambda nid, attrs: attrs.get('salary', 0) >= 90000))

print(f"\n📊 BEFORE loading initial state:")
print(f"   👔 Senior employees: {current_seniors}")
print(f"   🏠 Remote workers: {current_remote}")
print(f"   💰 High earners (≥$90k): {current_high_earners}")

# Try to load the initial state
print(f"\n⏪ Loading initial state {initial_hash[:12]}...")
try:
    success = g.load_state(initial_hash)
    if success:
        print(f"✅ Successfully loaded initial state!")
        
        # Check the data after loading
        loaded_seniors = len(g.filter_nodes({'role': 'Senior'}))
        loaded_remote = len(g.filter_nodes({'is_remote': True}))
        loaded_high_earners = len(g.filter_nodes(lambda nid, attrs: attrs.get('salary', 0) >= 90000))
        
        print(f"\n📊 AFTER loading initial state:")
        print(f"   👔 Senior employees: {loaded_seniors}")
        print(f"   🏠 Remote workers: {loaded_remote}")
        print(f"   💰 High earners (≥$90k): {loaded_high_earners}")
        
        print(f"\n📈 State comparison:")
        print(f"   👔 Seniors: {current_seniors} → {loaded_seniors} ({loaded_seniors - current_seniors:+d})")
        print(f"   🏠 Remote: {current_remote} → {loaded_remote} ({loaded_remote - current_remote:+d})")
        print(f"   💰 High earners: {current_high_earners} → {loaded_high_earners} ({loaded_high_earners - current_high_earners:+d})")
        
    else:
        print(f"❌ Failed to load state")
        
except Exception as e:
    print(f"❌ Error: {e}")

print(f"\n💾 Updated storage stats:")
updated_stats = g.get_storage_stats()
for key, value in updated_stats.items():
    print(f"   {key}: {value}")

🎯 Demonstrating complete state loading workflow:

📊 BEFORE loading initial state:
   👔 Senior employees: 100
   🏠 Remote workers: 7500
   💰 High earners (≥$90k): 120

⏪ Loading initial state 683eaad4aa34...
✅ Successfully loaded initial state!

📊 AFTER loading initial state:
   👔 Senior employees: 100
   🏠 Remote workers: 7500
   💰 High earners (≥$90k): 120

📈 State comparison:
   👔 Seniors: 100 → 100 (+0)
   🏠 Remote: 7500 → 7500 (+0)
   💰 High earners: 120 → 120 (+0)

💾 Updated storage stats:
   pooled_nodes: 31628
   edge_refs_tracked: 1500
   branches: 1
   total_states: 3
   node_refs_tracked: 31628
   pooled_edges: 1500

📊 AFTER loading initial state:
   👔 Senior employees: 100
   🏠 Remote workers: 7500
   💰 High earners (≥$90k): 120

📈 State comparison:
   👔 Seniors: 100 → 100 (+0)
   🏠 Remote: 7500 → 7500 (+0)
   💰 High earners: 120 → 120 (+0)

💾 Updated storage stats:
   pooled_nodes: 31628
   edge_refs_tracked: 1500
   branches: 1
   total_states: 3
   node_refs_tracked: 3162

In [9]:
# 🚀 Testing New Lazy-Loaded Properties (FIXED!)

# Note: Restart kernel to get updated Graph class
import sys
sys.path.insert(0, '/Users/michaelroth/Documents/Code/gli/python')
import gli

# Create a fresh graph to test the new properties
print("🚀 Creating fresh graph to test lazy-loaded properties:")
fresh_g = gli.Graph(backend='rust')

print(f"\n📚 fresh_g.states (lazy-loaded):")
states = fresh_g.states
for key, value in states.items():
    if key != 'auto_states':  # Skip the deque for cleaner output
        print(f"   {key}: {value}")

print(f"\n🌿 fresh_g.branches (lazy-loaded):")
branches = fresh_g.branches
for name, hash_val in branches.items():
    print(f"   {name}: {hash_val}")

# Add some data and test again
fresh_g.add_node('test1', department='Engineering', role='Junior')
fresh_g.add_node('test2', department='Marketing', role='Senior')
fresh_g.add_edge('test1', 'test2', relationship='collaborates')

# Commit and create branches
commit1 = fresh_g.commit('Initial data')
print(f"\n💾 Committed: {commit1}")

# Test filtering with new graph
marketing_folks = fresh_g.filter_nodes({'department': 'Marketing'})
seniors = fresh_g.filter_nodes({'role': 'Senior'})
print(f"\n🔍 Filtering test:")
print(f"   Marketing: {len(marketing_folks)} nodes")
print(f"   Seniors: {len(seniors)} nodes")

# Create a branch
fresh_g.create_branch('experiment', commit1)

print(f"\n📊 Final state:")
print(f"   States: {fresh_g.states['total_states']} total states")
print(f"   Branches: {fresh_g.branches}")

print(f"\n✅ Key improvements:")
print(f"   • g.states: lazy-loaded dict (no method calls needed)")
print(f"   • g.branches: lazy-loaded dict (replaces list_branches())")
print(f"   • Always in sync with Rust backend")
print(f"   • Properties computed on demand")
print(f"   • filter_nodes() supports both lambdas and attribute dicts")

🚀 Creating fresh graph to test lazy-loaded properties:

📚 fresh_g.states (lazy-loaded):
   total_states: 1
   pooled_nodes: 0
   pooled_edges: 0
   node_refs_tracked: 0
   edge_refs_tracked: 0
   current_hash: initial
   state_hashes: ['initial']
   branches_count: 1

🌿 fresh_g.branches (lazy-loaded):
   main: initial

💾 Committed: b58dbf422e7ee754

🔍 Filtering test:
   Marketing: 1 nodes
   Seniors: 1 nodes

📊 Final state:
   States: 2 total states
   Branches: {'main': 'initial', 'experiment': 'b58dbf422e7ee754'}

✅ Key improvements:
   • g.states: lazy-loaded dict (no method calls needed)
   • g.branches: lazy-loaded dict (replaces list_branches())
   • Always in sync with Rust backend
   • Properties computed on demand
   • filter_nodes() supports both lambdas and attribute dicts


In [10]:
g.nodes.items()[0:5]

[('node_66b4a24e',
  Node(id='node_66b4a24e', attributes={'salary': 63258, 'name': 'Employee_18968', 'is_remote': True, 'employee_id': 18968, 'department': 'Marketing', 'performance_score': 3.2, 'role': 'Junior'})),
 ('node_e3447fd0',
  Node(id='node_e3447fd0', attributes={'salary': 71218, 'performance_score': 3.5, 'is_remote': False, 'name': 'Employee_26633', 'department': 'Marketing', 'employee_id': 26633, 'role': 'Junior'})),
 ('node_2c9b3f4a',
  Node(id='node_2c9b3f4a', attributes={'salary': 68941, 'name': 'Employee_24589', 'department': 'Sales', 'performance_score': 3.8, 'role': 'Junior', 'employee_id': 24589, 'is_remote': False})),
 ('node_7359686d',
  Node(id='node_7359686d', attributes={'employee_id': 24550, 'department': 'Sales', 'role': 'Junior', 'name': 'Employee_24550', 'performance_score': 3.0, 'salary': 64820, 'is_remote': False})),
 ('node_c801e277',
  Node(id='node_c801e277', attributes={'department': 'Marketing', 'employee_id': 22397, 'role': 'Junior', 'is_remote': Fal

In [11]:
g.load_state(g.states['state_hashes'][-2])

True

In [12]:
g.nodes.items()[0:5]

[('node_89ce275d',
  Node(id='node_89ce275d', attributes={'salary': 66523, 'department': 'Sales', 'employee_id': 27055, 'performance_score': 3.8, 'name': 'Employee_27055', 'role': 'Junior', 'is_remote': False})),
 ('node_1cf06c4b',
  Node(id='node_1cf06c4b', attributes={'is_remote': False, 'performance_score': 3.9, 'role': 'Junior', 'employee_id': 526, 'department': 'Sales', 'name': 'Employee_526', 'salary': 57658})),
 ('node_ad588d9a',
  Node(id='node_ad588d9a', attributes={'name': 'Employee_10171', 'role': 'Junior', 'salary': 60237, 'is_remote': False, 'performance_score': 3.8, 'employee_id': 10171, 'department': 'Sales'})),
 ('node_782d8338',
  Node(id='node_782d8338', attributes={'salary': 75940, 'role': 'Junior', 'performance_score': 3.1, 'name': 'Employee_11759', 'employee_id': 11759, 'is_remote': False, 'department': 'Marketing'})),
 ('node_cf19fa0d',
  Node(id='node_cf19fa0d', attributes={'name': 'Employee_13553', 'employee_id': 13553, 'department': 'Marketing', 'salary': 66373

In [13]:
g.save_state("Final commit after testing")

'd9390fc131145899'

In [14]:
g.states

{'total_states': 4,
 'pooled_nodes': 31628,
 'pooled_edges': 1500,
 'node_refs_tracked': 31628,
 'edge_refs_tracked': 1500,
 'current_hash': 'd9390fc131145899',
 'state_hashes': ['d9390fc131145899', 'ca554ab1d40bc329', '683eaad4aa340052'],
 'auto_states': ['683eaad4aa340052', 'ca554ab1d40bc329', 'd9390fc131145899'],
 'branches_count': 1}

In [15]:
# 🎯 Final GLI API: save_state vs commit

print("🎯 NEW: save_state method (more intuitive than commit)")

# Create a fresh graph to demonstrate
demo_g = gli.Graph(backend='rust')
demo_g.add_node('alice', role='Engineer', team='Backend')
demo_g.add_node('bob', role='Designer', team='Frontend')

# Use the new save_state method
state1 = demo_g.save_state("Initial team setup")
print(f"✅ Saved state: {state1}")

# Add more data
demo_g.add_node('charlie', role='Manager', team='Backend')
state2 = demo_g.save_state("Added management layer")
print(f"✅ Saved state: {state2}")

# Test backward compatibility
demo_g.add_edge('alice', 'charlie', relationship='reports_to')
state3 = demo_g.commit("Using legacy commit method")  # Still works!
print(f"✅ Legacy commit: {state3}")

print(f"\n📊 Final state overview:")
final_states = demo_g.states
final_branches = demo_g.branches
print(f"   Total states: {final_states['total_states']}")
print(f"   State hashes: {final_states['state_hashes']}")
print(f"   Branches: {final_branches}")

# Test state loading
print(f"\n⏪ Testing state loading:")
print(f"Current nodes: {len(demo_g.nodes)}")
demo_g.load_state(state1)  # Go back to first state
print(f"After loading state1: {len(demo_g.nodes)} nodes")
demo_g.load_state(state2)  # Go to second state
print(f"After loading state2: {len(demo_g.nodes)} nodes")

print(f"\n🎉 GLI API Summary:")
print(f"   • g.save_state(message) - primary state saving")
print(f"   • g.commit(message) - backward compatibility")
print(f"   • g.load_state(hash) - time travel to any state")
print(f"   • g.states - comprehensive state info + hashes")
print(f"   • g.branches - branch dictionary")
print(f"   • g.filter_nodes/edges - dual interface (dict + lambda)")
print(f"   • All properties lazy-loaded from Rust backend!")

print(f"\n✅ GLI refactoring complete: maintainable, performant, robust!")

🎯 NEW: save_state method (more intuitive than commit)
✅ Saved state: 78f31ad65ee86645
✅ Saved state: a9bd3c955550b890
✅ Legacy commit: de3214257e1de298

📊 Final state overview:
   Total states: 4
   State hashes: ['78f31ad65ee86645', 'a9bd3c955550b890', 'de3214257e1de298']
   Branches: {'main': 'initial'}

⏪ Testing state loading:
Current nodes: 3
After loading state1: 2 nodes
After loading state2: 3 nodes

🎉 GLI API Summary:
   • g.save_state(message) - primary state saving
   • g.commit(message) - backward compatibility
   • g.load_state(hash) - time travel to any state
   • g.states - comprehensive state info + hashes
   • g.branches - branch dictionary
   • g.filter_nodes/edges - dual interface (dict + lambda)
   • All properties lazy-loaded from Rust backend!

✅ GLI refactoring complete: maintainable, performant, robust!
